# NLP Final Project - Colab only
Bu çalışmamızda Mayıs 1996'dan Temmuz 2014'e kadar listelenen tüm Elektronik ürünlerine ilişkin müşteri yorumlarını inceleyim düzenlemeler , iyi ve kötü yorumlara ayıracağız

Aşağıda aktif bir 1689188 satırlık yorum verisi'nin hangi dilde tespit eden bi kod var bende 2 saat civarı sürdü bilginiz olsun

In [1]:
#pip install autocorrect

In [2]:
#pip install langdetect

In [3]:
#pip install googletrans

In [4]:
#pip install TextBlob

In [5]:
#pip install WordCloud

In [6]:
#pip install transformers

In [7]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import time
import os
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import CountVectorizer
from autocorrect import spell
from textblob import TextBlob
from langdetect import detect
from googletrans import Translator
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from PIL import Image
from wordcloud import WordCloud, STOPWORDS
from transformers import pipeline

In [ ]:
dataset = "Electronics_5.json"

if os.path.isfile(dataset):
    df = pd.read_json("Electronics_5.json", lines=True)
else:
    url = r"http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz"
    df = pd.read_json(url, compression='gzip', lines=True)

display(df.head(5))

In [ ]:
df.sample(5)

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().sum().sort_values(ascending=False)

## <font color=blue> Feature Engineering

### <font color=#9e32a8> Gereksiz Verileri Silme İşlemi Yapıyoruz

In [ ]:
df=df.drop(['reviewTime'],axis=1)
df.head(1)

## <font color='red'> Boşluk Varsa Doldurmaya, Düzeltilecek Kısım Varsa Düzetmaye Başladık

In [ ]:
def algo_text(df):

  for col in df.columns:
    if df[col].dtype=='object':
      df[col] = df[col].str.lower()
      df[col] = df[col].str.replace('[^\w\s]', '', regex=True)
      df[col] = df[col].str.replace('\n', '')
      df[col] = df[col].str.replace('\d', '')
      df[col] = df[col].str.replace('\r', '')
      df[col] = df[col].str.replace('.', '')
      df[col] = df[col].str.replace(',', '')
    return df

df=algo_text(df)

In [ ]:
df.head()

In [ ]:
df['reviewText'][0]

In [ ]:
# reviewText sütunun'daki verilerin hangi dillerde yazıldığını tespit etme
def detect_veri(reviewText):
    try:
        return detect(reviewText)
    except:
        return 'unknown'

start_time = time.time()
df['language']=df['reviewText'].apply(detect_veri)
end_time = time.time()


elapsed_time = end_time - start_time
print(f"1689188 satır için geçen süre: {elapsed_time} saat")

In [ ]:
sns.countplot(x=df['language']);

In [ ]:
df=df[df['language']=='en']
df.drop(index= df[df['language']!='en'].index, inplace=True)

In [ ]:
sns.countplot(x=df['overall']);

In [ ]:
df['sentiment']=df['overall']
df['sentiment']=df['sentiment'].replace([5,4],'olumlu')
df['sentiment']=df['sentiment'].replace([1,2],'olumsuz')
df['sentiment']=df['sentiment'].replace([3],'notr')

## <font color=#3ede0d> Sentiment Analysis

### <font color=#f0be54> Yorumların olumlumu ya da olumsuzmu olduğunu tespit etme

In [ ]:
classifier = pipeline('sentiment-analysis')

In [ ]:
data = df['reviewText']

In [ ]:
def get_transformers_sentiment(text):
  result = classifier(text)
  return result [0]['label']

In [ ]:
#df['sentiment'] = data.apply(get_transformers_sentiment)
print(df[['reviewText', 'sentiment']].head())

In [ ]:
df.head()

In [ ]:
df=df[['overall','reviewText','sentiment']]

In [ ]:
df=df[(df['sentiment']=='olumlu')|(df['sentiment']=='notr')|(df['sentiment']=='olumsuz')]

In [ ]:
df.reset_index(drop=True,inplace=True)

In [ ]:
x=df['reviewText']
y=df['sentiment']

In [ ]:
vect=CountVectorizer()
x=vect.fit_transform(x)

In [ ]:
x=pd.get_dummies(x,drop_first=True)

In [ ]:
def algo_test(x,y):
    #Bütün modelleri tanımlıyoruz
    DTC=DecisionTreeClassifier()
    GNB=GaussianNB()
    BNB=BernoulliNB()
    MNB=MultinomialNB()
    RFC=RandomForestClassifier()
    GBC=GradientBoostingClassifier()
    KNC=KNeighborsClassifier()


    algos=[DTC,GNB,BNB,MNB,RFC,GBC,KNC]
    algo_names=['DecisionTreeClassifier','GaussianNB','BernoulliNB','MultinomialNB','RandomForestClassifier','GradientBoostingClassifier','KNeighborsClassifier']

    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.20,random_state=42)

    accuracy_score=[]
    confusion_matrix=[]
    classification_report=[]

    #Hata ve doğruluk oranlarını bir tablo haline getirmek için bir dataframe oluşturuluyor
    result=pd.DataFrame(columns=['accuracy_score','confusion_matrix','classification_report'])


    for algo in algos:
        p=algo.fit(x_train,y_train).predict(x_test)
        accuracy_score.append(accuracy_score(y_test, p))


    #result adlı tablo doğruluk ve hata oranlarını yerleştiriyor
    result.R_Squared=accuracy_score

    #tablo adlarını gösteriyor
    result.index = algo_names

    #oluşturduğum result tablosunu doğruluk oranına (r2_score) göre sıralayıp dönüyor
    rtable=result.sort_values('accuracy_score',ascending=False)
    return rtable

In [ ]:
algo_test(x,y)

In [ ]:
def wc(data,bgcolor):
    plt.figure(figsize=(10,10))
    mask=np.array(Image.open('cloud.png')) #resmi okumak içim veriyi diziye,rakama çevirdik
    wc=WordCloud(background_color=bgcolor,stopwords=STOPWORDS,mask=mask)#metinlerde sık kullanılan ancak anlamsal olarak önemli olmayan kelimeleri kaldırır
    #mask=mask yukarıda tanımladığımız maskeyi kullan
    wc.generate(''.join(data))
    plt.imshow(wc)
    plt.axis('off') # Bunla x ve y gözükmüyor

In [ ]:
pozitif=df[df['overall']==5 && df['sentiment']=olumlu]['reviewText']
pozitif=df[df['overall']==4 && df['sentiment']=olumlu]['reviewText']
notr=df[df['overall']==3 && df['sentiment']=notr]['reviewText']
negatif=df[df['overall']==2 && df['sentiment']=olumsuz]['reviewText']
negatif=df[df['overall']==1 && df['sentiment']=olumsuz]['reviewText']

In [ ]:
wc(pozitif,'green')

In [ ]:
wc(notr,'white')

In [ ]:
wc(negatif,'red')